In [1]:
%pip install jupysql prettytable --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext sql

In [3]:
# This creates the connection for the %%sql magic
%sql sqlite:///hng_ride.db

Connecting to 'sqlite:///hng_ride.db'

In [153]:
%%sql
PRAGMA table_info(drivers)

Running query in 'sqlite:///hng_ride.db'

cid,name,type,notnull,dflt_value,pk
0,driver_id,REAL,0,None,0
1,name,TEXT,0,None,0
2,city,TEXT,0,None,0
3,signup_date,TIMESTAMP,0,None,0
4,rating,REAL,0,None,0


Find the top 10 longest rides (by distance), including driver name, rider name, pickup/dropoff cities, and payment method.

In [31]:
%%sql
SELECT  rd.distance_km, 
        dr.name as driver_name, 
        rdr.name as rider_name,
        rd.pickup_city,
        rd.dropoff_city,
        pym.amount
FROM rides rd
JOIN drivers dr
ON rd.driver_id = dr.driver_id
JOIN riders rdr
ON rd.rider_id = rdr.rider_id
JOIN payments pym
ON rd.ride_id = pym.ride_id
WHERE 
    pym.amount > 0
    AND rd.request_time BETWEEN '2022-06-01 00:00' AND '2024-12-31 23:59'
ORDER BY rd.distance_km DESC
LIMIT 10

Running query in 'sqlite:///hng_ride.db'

distance_km,driver_name,rider_name,pickup_city,dropoff_city,amount
30.0,Driver_1774,Rider_292,Calgary,Los Angeles,74.33
30.0,Driver_1886,Rider_6131,Ottawa,Vancouver,79.21
29.99,Driver_862,Rider_2993,Los Angeles,Toronto,75.94
29.99,Driver_1574,Rider_9364,Boston,Toronto,82.78
29.99,Driver_1406,Rider_3294,New York,Toronto,84.76
29.99,Driver_1093,Rider_7646,Calgary,Boston,80.96
29.99,Driver_1832,Rider_7213,Ottawa,New York,71.63
29.99,Driver_1343,Rider_9618,Calgary,New York,79.87
29.99,Driver_1752,Rider_2994,Ottawa,Chicago,76.92
29.99,Driver_545,Rider_3255,Ottawa,San Francisco,76.32


How many riders who signed up in 2021 still took rides in 2024?

In [46]:
%%sql

SELECT COUNT(DISTINCT rd.rider_id) AS Active_Riders
FROM rides rd
JOIN riders rdr
ON rd.rider_id = rdr.rider_id
WHERE request_time BETWEEN '2024-01-01 00:00' AND '2024-12-31 23:59'
AND rdr.signup_date BETWEEN '2021-01-01 00:00' AND '2021-12-31 23:59'

Running query in 'sqlite:///hng_ride.db'

Active_Riders
2051


Compare quarterly revenue between 2021, 2022, 2023, and 2024. Which quarter had the biggest YoY growth?

In [59]:
%%sql
WITH QuarterlyRevenue AS (
    SELECT
        strftime('%Y', paid_date) AS SalesYear,
        (strftime('%m', paid_date) - 1) / 3 + 1 AS SalesQuarter,
        SUM(amount) AS QuarterRevenue
    FROM
        payments
    WHERE 
        amount > 0
        AND paid_date BETWEEN '2021-01-01 00:00' AND '2024-12-31 23:59'
    GROUP BY
        SalesYear,
        SalesQuarter
),
YOYGrowth AS (
    SELECT 
        SalesYear,
        SalesQuarter,
        QuarterRevenue,
        LAG(QuarterRevenue, 1, 0) OVER(
            PARTITION BY SalesQuarter
            ORDER BY SalesYear
        ) as PrevYearRevenue
    FROM
        QuarterlyRevenue
)
SELECT
    SalesYear,
    SalesQuarter,
    QuarterRevenue,
    PrevYearRevenue,
    (QuarterRevenue - PrevYearRevenue) * 100 / NULLIF(PrevYearRevenue, 0) AS YOYGrowthPercentage
FROM
    YOYGrowth
WHERE 
    PrevYearRevenue > 0
ORDER BY
    YOYGrowthPercentage DESC

Running query in 'sqlite:///hng_ride.db'

SalesYear,SalesQuarter,QuarterRevenue,PrevYearRevenue,YOYGrowthPercentage
2024,4,102246.26,98280.14,4.035525386919469
2024,3,103441.88,100238.16,3.196108148832741
2022,4,103944.7,101070.2,2.8440628394917593
2022,1,102518.55,100416.21,2.0936261187312253
2023,2,102557.18,102263.56,0.28712084734777016
2022,3,106099.28,106038.77,0.057064034220686226
2023,1,100544.36,102518.55,-1.925690521373939
2022,2,102263.56,105637.47,-3.1938572553848585
2024,2,97725.02,102557.18,-4.711674014437594
2024,1,95315.63,100544.36,-5.200420988308042


For each driver, calculate their average monthly rides since signup. Who are the top 5 drivers with the highest consistency (most rides per active month)?

In [93]:
%%sql
WITH DriverActiveMonth AS (
    SELECT
        driver_id,
        name,
        (strftime('%Y', '2024-12-31') - strftime('%Y', signup_date)) * 12 +
        (strftime('%m', '2024-12-31') - strftime('%m', signup_date)) + 1 AS total_months_active
    FROM
        drivers
),
DriverTotalRides AS (
    SELECT 
        rd.driver_id,
        COUNT(rd.ride_id) AS total_rides_completed
    FROM rides rd
    JOIN payments pym
    ON rd.ride_id = pym.ride_id
    WHERE pym.amount > 0
    AND rd.request_time BETWEEN '2021-06-01' AND '2024-12-31'
    GROUP BY rd.driver_id
)
SELECT 
    d.name,
    dt.total_rides_completed,
    d.total_months_active,
    (CAST(dt.total_rides_completed AS REAL) / d.total_months_active) AS rides_per_active_month
FROM DriverActiveMonth d
JOIN DriverTotalRides dt
ON dt.driver_id = d.driver_id
ORDER BY rides_per_active_month DESC
LIMIT 5

Running query in 'sqlite:///hng_ride.db'

name,total_rides_completed,total_months_active,rides_per_active_month
Driver_219,31,13,2.3846153846153846
Driver_1005,31,14,2.2142857142857144
Driver_1029,28,13,2.1538461538461537
Driver_363,26,14,1.8571428571428572
Driver_1355,26,14,1.8571428571428572


In [83]:
%%sql
    SELECT 
        rd.driver_id,
        COUNT(rd.ride_id) AS total_rides_completed
    FROM rides rd
    JOIN payments pym
    ON rd.ride_id = pym.ride_id
    GROUP BY rd.driver_id

Running query in 'sqlite:///hng_ride.db'

driver_id,total_rides_completed
1,24
2,17
3,16
4,23
5,12
6,24
7,24
8,24
9,18
10,23


In [82]:
%%sql
SELECT
        driver_id,
        name,
        (strftime('%Y', '2024-12-31') - strftime('%Y', signup_date)) * 12 +
        (strftime('%m', '2024-12-31') - strftime('%m', signup_date)) + 1 AS total_months_active
    FROM
        drivers

Running query in 'sqlite:///hng_ride.db'

driver_id,name,total_months_active
1.0,Driver_1,47
2.0,Driver_2,36
3.0,Driver_3,41
4.0,Driver_4,14
5.0,Driver_5,44
6.0,Driver_6,27
7.0,Driver_7,47
8.0,Driver_8,37
9.0,Driver_9,23
10.0,Driver_10,47


Calculate the cancellation rate per city and identify which city had the highest cancellation rate?

In [107]:
%%sql
SELECT 
    pickup_city,
    COUNT(rider_id) as total_rides,
    SUM(CASE WHEN status = 'cancelled' THEN 1 ELSE 0 END) as cancelled_rides,
    CAST(SUM(CASE WHEN status = 'cancelled' THEN 1 ELSE 0 END) AS REAL) / COUNT(rider_id) AS cancellation_rate
FROM
    rides
WHERE
    request_time BETWEEN '2021-06-01 00:00:00' AND '2024-12-31 23:59:59'
GROUP BY
    pickup_city
ORDER BY
    cancellation_rate DESC
LIMIT 1

Running query in 'sqlite:///hng_ride.db'

pickup_city,total_rides,cancelled_rides,cancellation_rate
Chicago,4458,859,0.1926873037236429


Identify riders who have taken more than 10 rides but never paid with cash.

In [136]:
%%sql
SELECT 
    rd.rider_id,
    rdr.name,
    COUNT(rd.ride_id) total_ride
FROM rides rd
JOIN payments pym
ON rd.ride_id = pym.ride_id
JOIN riders rdr
ON rd.rider_id = rdr.rider_id
WHERE
    pym.amount > 0
    AND rd.request_time BETWEEN '2021-06-01 00:00:00' AND '2024-12-31 23:59:59'
GROUP BY
    rd.rider_id 
HAVING
    COUNT(rd.ride_id) > 10
    AND SUM(CASE WHEN pym.method = 'cash' THEN 1 ELSE 0 END) = 0;

Running query in 'sqlite:///hng_ride.db'

rider_id,name,total_ride
7823,Rider_7823,12


Find the top 3 drivers in each city by total revenue earned between June 2021 and Dec 2024. If a driver has multiple cities, count revenue where they picked up passengers in that city.

In [ ]:
%%sql
WITH DriverRevenueByCity AS (
    SELECT
        rd.pickup_city,
        dr.name AS driver_name,
        SUM(p.amount) AS total_revenue
    FROM
        rides rd
    JOIN payments p 
    ON rd.ride_id = p.ride_id
    JOIN drivers dr 
    ON rd.driver_id = dr.driver_id
    WHERE
        p.amount > 0
        AND rd.request_time BETWEEN '2021-06-01 00:00:00' AND '2024-12-31 23:59:59'
    GROUP BY
        rd.pickup_city, dr.name
),
RankedDrivers AS (
    SELECT
        pickup_city,
        driver_name,
        total_revenue,
        ROW_NUMBER() OVER (
            PARTITION BY pickup_city
            ORDER BY total_revenue DESC
        ) AS rank
    FROM
        DriverRevenueByCity
)
SELECT
    pickup_city,
    driver_name,
    total_revenue,
    rank
FROM
    RankedDrivers
WHERE
    rank <= 3
ORDER BY
    pickup_city, rank;


Running query in 'sqlite:///hng_ride.db'

pickup_city,driver_name,total_revenue,rank
Boston,Driver_1176,448.4,1
Boston,Driver_286,326.58,2
Boston,Driver_1141,315.88,3
Calgary,Driver_1980,476.91,1
Calgary,Driver_1059,346.86,2
Calgary,Driver_404,338.8,3
Chicago,Driver_413,449.45,1
Chicago,Driver_1410,421.9,2
Chicago,Driver_1941,331.53,3
L.A,Driver_1241,128.82999999999998,1


Management wants to know the top 10 drivers that are qualified to receive bonuses using the criteria below;
at least 30 rides completed,
an average rating ≥ 4.5, and
a cancellation rate under 5%.


In [166]:
%%sql
WITH DriverRideStats AS (
    SELECT
        driver_id,
        COUNT(ride_id) AS total_rides_attempted,
        SUM(CASE WHEN status = 'cancelled' THEN 1 ELSE 0 END) AS total_cancelled
    FROM
        rides
    WHERE
        request_time BETWEEN '2021-06-01 00:00:00' AND '2024-12-31 23:59:59'
    GROUP BY
        driver_id
),
DriverCompletedStats AS (
    SELECT
        rd.driver_id,
        COUNT(rd.ride_id) AS total_completed_rides
    FROM
        rides rd
    JOIN
        payments p ON rd.ride_id = p.ride_id
    WHERE
        p.amount > 0
        AND rd.request_time BETWEEN '2021-06-01 00:00:00' AND '2024-12-31 23:59:59'
    GROUP BY
        rd.driver_id
)
SELECT
    dr.name,
    dr.rating AS average_rating,
    cs.total_completed_rides,
    (CAST(rs.total_cancelled AS REAL) * 100.0 / rs.total_rides_attempted) AS cancellation_rate_percent
FROM
    drivers dr
JOIN
    DriverRideStats rs ON dr.driver_id = rs.driver_id
JOIN
    DriverCompletedStats cs ON dr.driver_id = cs.driver_id
WHERE
    cs.total_completed_rides >= 30
    AND dr.rating >= 4.5
    AND (CAST(rs.total_cancelled AS REAL) * 100.0 / rs.total_rides_attempted) < 5.0
ORDER BY
    cs.total_completed_rides DESC
LIMIT 10; 

Running query in 'sqlite:///hng_ride.db'

name,average_rating,total_completed_rides,cancellation_rate_percent
Driver_1005,4.8,31,0.0
Driver_1181,4.6,31,3.125
